In [1]:
import numpy as np 

A = np.array([[1, 2, 3], [4, 5, 6]]).astype(np.float64)
B = np.array([[7, 8], [9, 10], [11, 12]]).astype(np.float64)
A,B

(array([[1., 2., 3.],
        [4., 5., 6.]]),
 array([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]))

In [2]:


A1 = np.array([[1, 2, 3], [4, 5, 6]]).astype(np.int32)
B1 = np.array([[7, 8], [9, 10], [11, 12]]).astype(np.int32)
A1,B1

(array([[1, 2, 3],
        [4, 5, 6]], dtype=int32),
 array([[ 7,  8],
        [ 9, 10],
        [11, 12]], dtype=int32))

## NUMPY Matrix Multiplication (Naive)

In [3]:
# define naive matrix multiplication in numpy
import numpy as np

def matrix_multiply_np(A, B):
    result = np.zeros((A.shape[0], B.shape[1]))

    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                result[i][j] += A[i][k] * B[k][j]
    return result


In [4]:

%%time

# float64 matrices
matrix_multiply_np(A, B)

CPU times: user 18 µs, sys: 15 µs, total: 33 µs
Wall time: 34.8 µs


array([[ 58.,  64.],
       [139., 154.]])

In [5]:

%%time

# int32 matrices
matrix_multiply_np(A1, B1)

CPU times: user 19 µs, sys: 15 µs, total: 34 µs
Wall time: 37.2 µs


array([[ 58.,  64.],
       [139., 154.]])

## NUMBA Matrix Multiplication

In [6]:
import numba 

@numba.jit(nopython=True)
def matrix_multiply_numba(A, B):
    result = np.zeros((A.shape[0], B.shape[1]))

    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                result[i][j] += A[i][k] * B[k][j]
    return result



In [7]:
%%time

# float64 matrices 1st time
matrix_multiply_numba(A, B)

CPU times: user 1.29 s, sys: 1.08 s, total: 2.36 s
Wall time: 621 ms


array([[ 58.,  64.],
       [139., 154.]])

In [8]:
%%time

# float64 matrices 2nd time
matrix_multiply_numba(A, B)

CPU times: user 9 µs, sys: 7 µs, total: 16 µs
Wall time: 17.9 µs


array([[ 58.,  64.],
       [139., 154.]])

In [9]:
%%time

# int32 matrices 1st time
matrix_multiply_numba(A1, B1)

CPU times: user 207 ms, sys: 0 ns, total: 207 ms
Wall time: 205 ms


array([[ 58.,  64.],
       [139., 154.]])

In [10]:
%%time

# int32 matrices 2nd time
matrix_multiply_numba(A1, B1)

CPU times: user 18 µs, sys: 0 ns, total: 18 µs
Wall time: 20.5 µs


array([[ 58.,  64.],
       [139., 154.]])

In [11]:
%%time

# float64 matrices 3rd time
matrix_multiply_numba(A, B)

CPU times: user 33 µs, sys: 0 ns, total: 33 µs
Wall time: 35.8 µs


array([[ 58.,  64.],
       [139., 154.]])

## CYTHON Matrix Multiplication Version 1

In [12]:
%load_ext Cython

In [13]:
%%cython
import numpy as np
import cython

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.locals(A=cython.double[:, :], B=cython.double[:, :], result=cython.double[:,:], i=cython.int, j=cython.int, k=cython.int)
def matrix_multiply_cython(A, B):
    result = np.zeros((A.shape[0], B.shape[1]))

    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                result[i][j] += A[i][k] * B[k][j]
    return np.asarray(result)



In [14]:
%time 

# float64 matrices
matrix_multiply_cython(A, B)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


array([[ 58.,  64.],
       [139., 154.]])

In [15]:
%time 

# int32 matrices
matrix_multiply_cython(A1, B1)

CPU times: user 4 µs, sys: 2 µs, total: 6 µs
Wall time: 25.5 µs


ValueError: Buffer dtype mismatch, expected 'double' but got 'int'

## TAICHI Matrix Multiplication

In [16]:
import taichi as ti

ti.init(arch=ti.cpu)  # Use CPU by default

def matrix_multiply_taichi(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    C = ti.field(shape=(A.shape[0], B.shape[1]), dtype=ti.f64)

    # Convert the numpy arrays to Taichi ndarrays
    A_ti = ti.field(shape=A.shape, dtype=ti.f64)
    B_ti = ti.field(shape=B.shape, dtype=ti.f64)
    A_ti.from_numpy(A)
    B_ti.from_numpy(B)
    sum = ti.field(dtype=ti.f64, shape=())

    @ti.kernel
    def _taichi_compute():
        ti.loop_config(serialize=True)
        for i in range(A.shape[0]):
            for j in range(B.shape[1]):
                sum[None] = 0.0
                for k in range(A.shape[1]):
                    sum[None] += A_ti[i, k] * B_ti[k, j]
                C[i,j] = sum[None]

    _taichi_compute()
    return C.to_numpy()


[Taichi] version 1.7.0, llvm 15.0.4, commit 2fd24490, linux, python 3.10.12


[I 12/30/23 04:01:16.705 11531] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=x64


In [17]:
%time 

# float64 matrices
matrix_multiply_taichi(A, B)


CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 6.68 µs


array([[ 58.,  64.],
       [139., 154.]])

In [18]:
%time 

# int32 matrices
matrix_multiply_taichi(A1, B1)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.44 µs


array([[ 58.,  64.],
       [139., 154.]])